# Markov Decision Process (MDP) 구현


In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np
import gym
import matplotlib.pyplot as plt


C:\Users\night\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\night\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\night\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\night\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [2]:
%matplotlib inline

In [3]:
env = gym.make('CartPole-v0')

In [6]:
gamma = 0.99

def discount_rewards(r):
    # 보상의 1-Dimension 실수 배열을 취해서 Discounted Reward를 계산
    discounted_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(range(0,r.size)):
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r

class agent():
    def __init__(self, lr, s_size, a_size, h_size):
        # 네트워크의 피드포워드 부분, 에이전트는 State를 받아서 Action을 출력
        self.state_in = tf.placeholder(shape=[None, s_size], dtype=tf.float32)
        hidden = slim.fully_connected(self.state_in, h_size, biases_initializer=None,\
                                     activation_fn=tf.nn.relu)
        self.output = slim.fully_connected(hidden,a_size, biases_initializer=None,\
                                          activation_fn=tf.nn.softmax)
        self.chosen_action = tf.argmax(self.output,1)
        
        #학습 과정 구현. Cost를 계산하기 위해 Reward와 Action을 네트워크에 피드하고, 네트워크를 업데이트한다.
        self.reward_holder = tf.placeholder(shape=[None], dtype=tf.float32)
        self.action_holder = tf.placeholder(shape=[None], dtype=tf.int32)
        
        self.indexes = tf.range(0, tf.shape(self.output)[0]) * tf.shape(self.output[1]) + self.action_holder
        self.responsible_outputs = tf.gather(tf.reshape(self.output, [-1]), self.indexes)
        
        self.loss = -tf.reduce_mean(tf.log(self.responsible_outputs)*\
                                   self.reward_holder)
        
        tvars = tf.trainable_variables()
        self.gradient_holders = []
        for idx,var in enumerate(tvars):
            placeholder = tf.placeholder(tf.float32, name=str(idx) + '_holder')
            self.gradient_holders.append(placeholder)
            
        self.gradients = tf.gradients(self.loss, tvars)
        
        optimizer = tf.train.AdamOptimizer(learning_rate=lr)
        self.update_batch = optimizer.apply_gradients(zip(self.gradient_holders, tvars))
        
        

In [ ]:
# 텐서플로 그래프를 리셋한다.
tf.reset_default_graph()

# 에이전트를 로드한다.
myAgent = agent(lr=1e-2, s_size=4, a_size=2, h_size=8)

# 에이전트를 학습시킬 총 에피소드의 수를 설정한다.
total_episodes = 5000
max_ep = 999
update_frequency = 5


init = tf.global_variables_initializer()

# 텐서플로 그래프를 론칭한다.
with tf.Session() as sess:
    sess.run(init)
    i = 0
    total_reward = []
    total_length = []
    
    gradBuffer = sess.run(tf.trainable_variables())
    for ix,grad in enumerate(gradBuffer):
        gradBuffer[ix] = grad * 0
    
    while i < total_episodes:
        s = env.reset()
        running_reward = 0
        ep_history = []
        for j in range(max_ep):
            # 네트워크 출력에서 확률적으로 액션을 선택한다.
            a_dist = sess.run(myAgent.output, feed_dict={myAgent.state_in:[s]})
            a = np.random.choice(a_dist[0], p=a_dist[0])
            a = np.argmax(a_dist == a)
            
            # 주어진 밴딧에 대해 액션을 취한 것에 대한 보상을 얻는다.
            s1, r, d, _ = env.step(a)
            ep_history.append([s,a,r,s1])
            s = s1
            running_reward += r
            if d == True:
                # 네트워크를 업데이트한다.
                ep_history = np.array(ep_history)
                ep_history[:,2] = discount_rewards(ep_history[:,2])
                feed_dict={myAgent.reward_holder:ep_history[:,2], 
                          myAgent.action_holder:ep_history[:,1],
                          myAgent.state_in:np.vstack(ep_history[:,0])}
                grads = sess.run(myAgent.gradients, feed_dict = feed_dict)
                for idx, grad in enumerate(grads):
                    gradBuffer[idx] += grad
                    
                if i % update_frequency == 0 and i != 0:
                    feed_dict = dictionary = dict(zip(myAgent.gradient_holders,\
                                                     gradBuffer))
                    _ = sess.run(myAgent.update_batch, feed_dict = feed_dict)
                    for ix, grad in enumerate(gradBuffer):
                        gradBuffer[ix] = grad * 0
                    
                total_reward.append(running_reward)
                total_length.append(j)
                break
                
                
        # 보상의 총계 업데이트
        if i % 100 == 0:
            print(np.mean(total_reward[-100:]))
        i += 1
                

11.0
20.79
22.56
29.27
43.32
49.29
64.95
95.41
128.19
151.17
158.93
164.89
153.82
171.09
181.64
183.7
191.11
190.35
